# Bayesian Linear Regression
Created by: Eunhyuk Shin\
Contact: silvershine157@kaist.ac.kr

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
X_SCALE = 3.0
W_SCALE = 2.0
BETA = 2.0 # = 1/sigma^2

def visualize_model(X, y, w_dict):
    plt.figure()
    ax = plt.subplot(1,2,1)
    x_line = np.array([-X_SCALE, X_SCALE])
    for key in w_dict:
        w = w_dict[key]
        y_line = w[0, 1]*x_line + w[0, 0]
        ax.plot(x_line, y_line, label=key)
    ax.scatter(X[:, 1], y[:, 0], marker='.', color='k')
    ax.set_aspect(aspect=1.0)
    ax.legend()
    ax.axis([-X_SCALE, X_SCALE, -X_SCALE, X_SCALE])
    ax = plt.subplot(1,2,2)
    for key in w_dict:
        w = w_dict[key]
        ax.scatter(w[0, 1], w[0, 0], label=key)
    ax.set_aspect(aspect=1.0)
    ax.legend()
    ax.axis([-W_SCALE, W_SCALE, -W_SCALE, W_SCALE])
    plt.show()

def make_data():
    N = 5
    X1 = np.random.randn(N)
    X = np.stack([np.ones(N), X1], axis=1) # (N, 2)
    w_true = np.array([[-.5, 0.3]]) # (1, 2)
    y = np.matmul(X, w_true.transpose()) + (1/np.sqrt(BETA))*np.random.randn(N, 1) # (N, 1)
    return X, y, w_true

X, y, w_true = make_data()
visualize_model(X, y, {"true": w_true})

In [ ]:
def mle(X, y):
    XT = X.transpose()
    w_mle = np.matmul(np.linalg.inv(np.matmul(XT, X)), np.matmul(XT, y)).transpose()
    return w_mle
w_mle = mle(X, y)
visualize_model(X, y, {"True": w_true, "MLE": w_mle})

In [ ]:
def get_density(m, S):
    beta = 0.5
    W0 = np.linspace(-W_SCALE, W_SCALE)
    W1 = np.linspace(-W_SCALE, W_SCALE)
    W0, W1 = np.meshgrid(W0, W1)
    W = np.stack([W0, W1], axis=2) # (G, G, 2)
    S_det = np.linalg.det(S)
    S_inv = np.linalg.inv(S)
    normalizer = np.sqrt((((2*np.pi)**2))*S_det)
    fac = np.einsum('...k,kl,...l->...', W-m, S_inv, W-m) # (G, G)
    log_density = -fac/2.0 - np.log(normalizer)
    density = np.exp(log_density)
    return density
    
def visualize_bayesian(m, S):
    plt.figure()
    ax = plt.subplot(1,2,1)
    n_sampling = 50
    for _ in range(n_sampling):
        w = np.random.multivariate_normal(m, S).reshape(1, 2)
        x_line = np.array([-X_SCALE, X_SCALE])
        y_line = w[0, 1]*x_line + w[0, 0]
        ax.plot(x_line, y_line, 'y--')
    ax.set_aspect(aspect=1.0)
    ax.axis([-X_SCALE, X_SCALE, -X_SCALE, X_SCALE])
    ax = plt.subplot(1,2,2)
    density = get_density(m, S)
    ax.imshow(np.flip(density, axis=1).transpose())
    plt.show()
    
m0 = np.array([0, 0])
S0 = 1.0*np.eye(2)
visualize_bayesian(m0, S0)

In [ ]:
def get_posterior(m0, S0, X, y):
    S0_inv = np.linalg.inv(S0)
    SN = np.linalg.inv(S0_inv + BETA*np.matmul(X.transpose(), X))
    mN = np.matmul(SN, np.matmul(S0_inv, m0) + BETA*np.matmul(X.transpose(), y).reshape(2))
    return mN, SN

mN, SN = get_posterior(m0, S0, X, y)
visualize_model(X, y, {"true": w_true})
visualize_bayesian(mN, SN)